# Boolean Query

In this notebook, we will implement a Boolean information retrieval model by Elasticsearch.  
We have these steps: <br>

**1- Connect to the Elasticsearch Cluster and Create an Index <br>
2- Indexing Documents <br>
3- Boolean Retrieval (Fill where ever it says #TODO in this part)** <br>

Let me know if you have any problems with this notebook or implementation. <br>
**Telegram**: [@Mohammad_Ardestani4](https://t.me/Mohammad_Ardestani4) <br>
**Email**: mjavad.ardestani00@gmial.com

In [1]:
# !pip install elasticsearch

In [2]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [7]:
import numpy as np
import pandas as pd

In [8]:
import json
import pickle

In [9]:
import os
import time

In [10]:
from datetime import datetime
from tqdm import tqdm 

## Ignore The Security Warnings
Here we ignore the security warnings. It's suggested that you don't run below cell until you ensure that all warnings are about security. 


In [11]:
import warnings
warnings.filterwarnings('ignore')

## Load Processed Data

In [13]:
df = pd.read_pickle('../results/preprocessed-phase02.pickle')

In [14]:
df.head()

,id,title,content,tags,date,url,category,preprocessed
0,0,اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا,\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آس...,"[اعلام زمان, قرعه‌کشی, قرعه‌کشی جام, قرعه‌کشی ...",3/15/2022 5:59:27 PM,https://www.farsnews.ir/news/14001224001005/اع...,sports,"{'گزارش': 1, 'خبرگزاری': 1, 'فارس': 1, 'کنفدرا..."
1,1,سجادی :حضور تماشاگران در لیگ برتر فوتبال تابع...,\nبه گزارش خبرگزاری فارس، سید حمید سجادی در حا...,"[سجادی, لیگ, فدراسیون, وزیر ورزش]",3/15/2022 5:30:07 PM,https://www.farsnews.ir/news/14001224000982/سج...,sports,"{'گزارش': 1, 'خبرگزاری': 1, 'فارس': 1, 'سید': ..."
2,2,محل برگزاری نشست‌های خبری سرخابی‌ها؛ مجیدی در ...,\nبه گزارش خبرگزاری فارس، نشست خبری پیش از مسا...,"[دربی 94, محل برگزاری, خبری سرخابی‌ها, مجیدی, ...",3/15/2022 5:20:01 PM,https://www.farsnews.ir/news/14001224000971/مح...,sports,"{'گزارش': 1, 'خبرگزاری': 1, 'فارس': 1, 'خبری':..."
3,3,ماجدی در نشست با صالحی امیری: امیدوارم در این ...,\nبه گزارش خبرگزاری فارس، سید رضا صالحی امیری...,"[کمیته امداد امام خمینی (ره), کمیته ملی المپیک...",3/15/2022 5:18:00 PM,https://www.farsnews.ir/news/14001224000964/ما...,sports,"{'گزارش': 1, 'خبرگزاری': 1, 'فارس': 1, 'سید': ..."
4,4,لیگ‌برتر بسکتبال|‌ نخستین پیروزی شهرداری گرگان...,\nبه گزارش خبرنگار ورزشی خبرگزاری فارس، در نخس...,"[بسکتبال, لیگ برتر بسکتبال, شهرداری گرگان, تیم...",3/15/2022 5:16:41 PM,https://www.farsnews.ir/news/14001224000947/لی...,sports,"{'گزارش': 1, 'خبرنگار': 1, 'ورزشی': 1, 'خبرگزا..."


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12202 entries, 0 to 12201
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            12202 non-null  int64 
 1   title         12202 non-null  object
 2   content       12202 non-null  object
 3   tags          12202 non-null  object
 4   date          12202 non-null  object
 5   url           12202 non-null  object
 6   category      12202 non-null  object
 7   preprocessed  12202 non-null  object
dtypes: int64(1), object(7)
memory usage: 762.8+ KB


In [17]:
df['preprocessed'].iloc[0].keys()

dict_keys(['گزارش', 'خبرگزاری', 'فارس', 'کنفدراسیون', 'فوتبال', 'آسیا', 'AFC', 'نامه', 'رسمی', 'فدراسیون', 'ایران', 'باشگاه', 'گیتی', 'پسند', 'زمان', 'قرعه\u200cکشی', 'جام', 'فوتسال', 'رسما', 'اعلام', 'اساس', '25', 'فروردین', 'ماه', '1401', 'مراسم', 'مالزی', 'برگزار', 'شد&شو', 'گیتی\u200cپسند', 'بعنوان', 'قهرمان', 'سال', '1400', 'مسابقات', 'تجربه', '3', 'دوره', 'حضور', 'فینال', 'عنوان', 'قهرمانی', 'مقام', 'دومی', 'بدست\u200cآورده', 'اس'])

## Making preprocessed column string

In [18]:
def join_spacewise(dictionary):
    result = ''
    for term in dictionary.keys():
        result += ' ' + term
        
    return result

In [19]:
join_spacewise(df['preprocessed'].iloc[0])

' گزارش خبرگزاری فارس کنفدراسیون فوتبال آسیا AFC نامه رسمی فدراسیون ایران باشگاه گیتی پسند زمان قرعه\u200cکشی جام فوتسال رسما اعلام اساس 25 فروردین ماه 1401 مراسم مالزی برگزار شد&شو گیتی\u200cپسند بعنوان قهرمان سال 1400 مسابقات تجربه 3 دوره حضور فینال عنوان قهرمانی مقام دومی بدست\u200cآورده اس'

In [18]:
for k, v in data['0'].items():
    print(k, ': ', v, sep='')

title: اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا
content: 
به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/



tags: ['اعلام زمان', 'قرعه\u200cکشی', 'قرعه\u200cکشی جام', 'قرعه\u200cکشی جام باشگاه\u200cهای فوتسال', 'ای اف سی', 'گیتی پسند']
date: 3/15/2022 5:59:27 PM
url: https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا
category: sports


In [21]:
df['preprocessed'] = df['preprocessed'].apply(join_spacewise)

## Connect to the Elasticsearch Cluster and Create an Index
After starting your Elasticsearch on your pc (`localhost:9200` is the default), we have to connect to it via the following piece of code.


In [22]:
index_name = 'news_index'

In [23]:
es = Elasticsearch("http://localhost:9200")

if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    
es.indices.create(index=index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'news_index'})

### Check the Cluster Status

In [24]:
dict(es.info())

{'name': 'ali',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '8xewuQfJQ-WkYfBiEGcwzg',
 'version': {'number': '8.2.3',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '9905bfb62a3f0b044948376b4f607f70a8a151b4',
  'build_date': '2022-06-08T22:21:36.455508792Z',
  'build_snapshot': False,
  'lucene_version': '9.1.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## Indexing Documents 

We can add our documents to created index in two ways:<br>
1. One by one in for loop. <br>
2. Use [Bulk API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html#:~:text=Bulk%20APIedit,can%20greatly%20increase%20indexing%20speed.), which performs multiple indexing or delete operations in a single API call. This reduces overhead and can significantly increase indexing speed.

###  for loop by es.index 
**You dont have to run this cell**. 

In [25]:
# for i in tqdm(range(len(data))):
#     es.index(index = index_name, id=i, document=data[str(i)])
#     i = i + 1

## Converting dataframe to dictionary

In [28]:
dict_df = df.to_dict('records')

In [29]:
dict_df[0]

{'id': 0,
 'title': 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا',
 'content': '\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان\xa0 قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/\n\n\n',
 'tags': ['اعلام زمان',
  'قرعه\u200cکشی',
  'قرعه\u200cکشی جام',
  'قرعه\u200cکشی جام باشگاه\u200cهای فوتسال',
  'ای اف سی',
  'گیتی پسند'],
 'date': '3/15/2022 5:59:27 PM',
 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا',
 'category': 'sports',
 'preprocessed': ' گزارش خبرگزاری فارس کنفدراسیون فوتبا

In [30]:
len(dict_df)

12202

## Converting dictionary to elastic search format

In [31]:
df.columns

Index(['id', 'title', 'content', 'tags', 'date', 'url', 'category',
       'preprocessed'],
      dtype='object')

In [48]:
def generator(df2):
    for c, row in enumerate(dict_df):
        yield {
            '_index': index_name,
            '_id': row.get('id', None),
            '_source': {
                'title': row.get('title', None),
                'content': row.get('content', None),
                'tags': row.get('tags', None),
                'date': row.get('date', None),
                'url': row.get('url', None),
                'category': row.get('category', None),
                'preprocessed': row.get('preprocessed', None),
            }
        }
    raise StopIteration

In [49]:
my_news = generator(dict_df)

In [50]:
my_news

<generator object generator at 0x7f85a26313c0>

In [51]:
print(json.dumps(next(my_news), indent=4, ensure_ascii=False).encode('utf8').decode())

{
    "_index": "news_index",
    "_id": 0,
    "_source": {
        "title": "اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا",
        "content": "\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/\n\n\n",
        "tags": [
            "اعلام زمان",
            "قرعه‌کشی",
            "قرعه‌کشی جام",
            "قرعه‌کشی جام باشگاه‌های فوتسال",
            "ای اف سی",
            "گیتی پسند"
        ],
        "date": "3/15/2022 5:59:27 PM",
        "url": "https://www.farsnews.ir/news/1400122

## Uploading data to index

###  Bulk API

In [52]:
try:
    res = helpers.bulk(es, generator(dict_df))
    print('working ...')
except Exception as e:
    pass

### check index

In [53]:
es.count(index = index_name)

ObjectApiResponse({'count': 12000, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Boolean Retrieval 

### Bool Query Structure

It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:

`must`: The clause (query) must appear in matching documents and will contribute to the score.
<br><br>
`filter`: The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching.
<br><br>
`should`: The clause (query) should appear in the matching document.
<br><br>
`must_not`: The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html). 

###  <span style="color:red"> TODO </span>
You should read about [match query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-top-level-params) and [match phrase query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html) then complete the below cell.

In [40]:
query= {
        "bool": {
          "should": [
              { 
                  #TODO: add a match query structure ==> use for normal words
              }, 
              
              { 
                  #TODO: add a match pharse query strucutre ==> use for words in <"">
              },
   
          ],
          "must_not": [
              {
                  #TODO: add a match or match pharse query structure ==> use for words after <!>
              }
          ],
        },
    }

### Search query

The `_source` field contains the original JSON document body that was passed at index time. The `_source` field itself is not indexed (and thus is not searchable), but it is stored so that it can be returned when executing fetch requests, like get or search.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html#search-api-response-body).

In [42]:
res = es.search(index=index_name, query=query, _source= ["url"])
res = dict(res)

### Results

In [ ]:
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])